In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import SAT1Start, ShallowConvNet, EEGNet
from shared.generators import SAT1DataGenerator
from shared.utilities import earlyStopping_cb
from sklearn.metrics import classification_report

%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [4]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [6]:
train_data = data.sel(
    participant=[
        "0021",
        "0022",
        "0023",
        "0024",
    ]
)
test_data = data.sel(participant=["0025"])

In [18]:
train_data

<xarray.Dataset>
Dimensions:      (channels: 30, samples: 157, epochs: 200, participant: 4,
                  labels: 4)
Coordinates:
  * channels     (channels) object 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 150 151 152 153 154 155 156
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * participant  (participant) object '0021' '0022' '0023' '0024'
  * labels       (labels) object 'confirmation' 'decision' 'encoding' 'response'
    stim         (participant, epochs) float64 2.0 1.0 1.0 1.0 ... 2.0 1.0 1.0
    resp         (participant, epochs) object 'resp_right' ... 'resp_left'
    rt           (participant, epochs) float64 1.388 1.388 0.47 ... 0.317 0.756
    cue          (participant, epochs) object 'AC' 'AC' 'SP' ... 'AC' 'SP' 'AC'
    movement     (participant, epochs) object 'stim_right' ... 'stim_left'
    trigger      (participant, epochs) object 'AC/stim_right/resp_right' ... ...
Data variables:
    data         (participant, epochs, labels, channels, samples) float64 -11...

In [182]:
train_np = train_data.to_array().to_numpy()
train_np.shape

(1, 4, 200, 4, 30, 157)

In [183]:
# Collapse participant + epoch dimensions into one and set label to first dimension
train_np = np.reshape(train_np, (-1,) + train_np.shape[3:])
train_np.shape

(800, 4, 30, 157)

In [179]:
nan_slices = np.all(np.isnan(train_np), axis=(2, 3))
# nan_slices = np.expand_dims(nan_slices, 2)

In [187]:
# Collapses first two dimensions
# filtered_data = train_np[~nan_slices]

filtered_data = train_np[~nan_slices].reshape(
    (-1, train_np.shape[1], train_np.shape[2], train_np.shape[3])
)
filtered_data.shape

(622, 4, 30, 157)

In [186]:
# TODO: Create lists of expected shape (622, 30, 157) containing data and (622, ) containing indices

IndexError: index 612 is out of bounds for axis 1 with size 4